## Predict Crypto-Price



#### Load data

In [1]:
#pip install python-binance  ta

In [2]:
import pandas as pd
import ta
import requests


In [3]:
#API = 2fd7d110c2283b5ebc80bc2b33cf63226b5e5cefd82696595f6bde8063fca87f


api_key = '2fd7d110c2283b5ebc80bc2b33cf63226b5e5cefd82696595f6bde8063fca87f'  

url = "https://min-api.cryptocompare.com/data/v2/histohour"   

parameters = {
    'fsym': 'BTC',       # from symbol
    'tsym': 'USD',       # to symbol
    'limit': 2000,        # number of days
    'api_key': api_key
}

response = requests.get(url, params=parameters)
data = response.json()

# Check if the response is OK
if data['Response'] == 'Success':
    ohlcv = data['Data']['Data']
    df = pd.DataFrame(ohlcv)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    
    print(df.head())
else:
    print("Error fetching data:", data['Message'])


                         high       low      open  volumefrom      volumeto  \
time                                                                          
2025-03-01 05:00:00  86253.57  85568.91  86173.31      538.19  4.623596e+07   
2025-03-01 06:00:00  85644.64  84853.53  85575.51      766.52  6.529094e+07   
2025-03-01 07:00:00  85046.16  84712.68  85045.67     1463.75  1.242902e+08   
2025-03-01 08:00:00  85312.73  84717.15  84717.19      418.38  3.563211e+07   
2025-03-01 09:00:00  85490.36  85108.81  85290.61      397.13  3.390820e+07   

                        close conversionType conversionSymbol  
time                                                           
2025-03-01 05:00:00  85575.51         direct                   
2025-03-01 06:00:00  85045.67         direct                   
2025-03-01 07:00:00  84717.19         direct                   
2025-03-01 08:00:00  85290.61         direct                   
2025-03-01 09:00:00  85367.68         direct                  

In [4]:
data.keys()

dict_keys(['Response', 'Message', 'HasWarning', 'Type', 'RateLimit', 'Data'])

In [5]:
data['Response']

'Success'

#### Prepare dataframe and add indicators

In [6]:
# 1. Add technical indicators

df['MA20'] = df['close'].rolling(window=20).mean()
df['RSI'] = ta.momentum.RSIIndicator(df['close']).rsi()
df['MACD'] = ta.trend.MACD(df['close']).macd()
df['MACD_signal'] = ta.trend.MACD(df['close']).macd_signal()


In [7]:
df.head(2)

,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol,MA20,RSI,MACD,MACD_signal
time,,,,,,,,,,,,
2025-03-01 05:00:00,86253.57,85568.91,86173.31,538.19,46235959.11,85575.51,direct,,NaN,NaN,NaN,NaN
2025-03-01 06:00:00,85644.64,84853.53,85575.51,766.52,65290936.36,85045.67,direct,,NaN,NaN,NaN,NaN


In [8]:
# Lag features (price from previous hour)
df['Lag_1'] = df['close'].shift(1)
df['Lag_2'] = df['close'].shift(2)

In [9]:
df.head(2)

,high,low,open,volumefrom,volumeto,close,conversionType,conversionSymbol,MA20,RSI,MACD,MACD_signal,Lag_1,Lag_2
time,,,,,,,,,,,,,,
2025-03-01 05:00:00,86253.57,85568.91,86173.31,538.19,46235959.11,85575.51,direct,,NaN,NaN,NaN,NaN,NaN,NaN
2025-03-01 06:00:00,85644.64,84853.53,85575.51,766.52,65290936.36,85045.67,direct,,NaN,NaN,NaN,NaN,85575.51,NaN


## Preprocess data

Handle null values

In [10]:
df.isna().sum()

high                 0
low                  0
open                 0
volumefrom           0
volumeto             0
close                0
conversionType       0
conversionSymbol     0
MA20                19
RSI                 13
MACD                25
MACD_signal         33
Lag_1                1
Lag_2                2
dtype: int64

In [11]:
#Drop null values (first 26 rows of the data)
df.dropna(inplace = True)

## Feature Engineering

In [12]:
df.drop('conversionSymbol', axis = 1, inplace = True)

#### Handle categorical data

In [13]:
df['conversionType'].nunique()

1

In [14]:
df['conversionType'] = df['conversionType'].map({'direct' : 1}).astype('i1')

In [15]:
df.head(2)

,high,low,open,volumefrom,volumeto,close,conversionType,MA20,RSI,MACD,MACD_signal,Lag_1,Lag_2
time,,,,,,,,,,,,,
2025-03-02 14:00:00,85892.38,85047.82,85702.44,1372.12,1.170603e+08,85104.24,1,85891.7015,39.217600,60.216278,150.560966,85702.44,85980.07
2025-03-02 15:00:00,88662.04,85067.08,85104.24,6871.29,6.000891e+08,87433.23,1,85990.3700,66.675576,187.674224,157.983618,85104.24,85702.44


#### Handle Datetime feature

In [16]:
#extract datetime features
df['day'] = df.index.day
df['year'] = df.index.year
df['month'] = df.index.month
df['day_name'] = df.index.day_name()

#### Save dataframe locally

In [17]:
df.to_csv("crypto_hourly_data.csv", index=False)

In [18]:
df.head()

,high,low,open,volumefrom,volumeto,close,conversionType,MA20,RSI,MACD,MACD_signal,Lag_1,Lag_2,day,year,month,day_name
time,,,,,,,,,,,,,,,,,
2025-03-02 14:00:00,85892.38,85047.82,85702.44,1372.12,1.170603e+08,85104.24,1,85891.7015,39.217600,60.216278,150.560966,85702.44,85980.07,2,2025,3,Sunday
2025-03-02 15:00:00,88662.04,85067.08,85104.24,6871.29,6.000891e+08,87433.23,1,85990.3700,66.675576,187.674224,157.983618,85104.24,85702.44,2,2025,3,Sunday
2025-03-02 16:00:00,91954.91,87352.89,87433.23,12097.35,1.088302e+09,91247.12,1,86277.0360,81.452069,589.637638,244.314422,87433.23,85104.24,2,2025,3,Sunday
2025-03-02 17:00:00,95106.04,90689.81,91247.12,8378.28,7.791416e+08,94130.91,1,86680.3800,86.372516,1127.892647,421.030067,91247.12,87433.23,2,2025,3,Sunday
2025-03-02 18:00:00,94445.48,92346.14,94130.91,4417.80,4.120838e+08,92790.10,1,87014.5385,76.244880,1429.789733,622.782000,94130.91,91247.12,2,2025,3,Sunday


#### Handle categorical datetime features

In [19]:
#import encoder library
from sklearn.preprocessing import LabelEncoder

In [20]:
encoder = LabelEncoder()   #intialize encoder
df['day_name'] = encoder.fit_transform(df['day_name'])
df['day'] = encoder.fit_transform(df['day'])
df['month'] =  encoder.fit_transform(df['month'])

In [21]:
df['year'].unique()

array([2025])

In [22]:
df['year'] = df['year'].map({2025 : 1}).astype('i1')

In [23]:
df.head(5)

,high,low,open,volumefrom,volumeto,close,conversionType,MA20,RSI,MACD,MACD_signal,Lag_1,Lag_2,day,year,month,day_name
time,,,,,,,,,,,,,,,,,
2025-03-02 14:00:00,85892.38,85047.82,85702.44,1372.12,1.170603e+08,85104.24,1,85891.7015,39.217600,60.216278,150.560966,85702.44,85980.07,1,1,0,3
2025-03-02 15:00:00,88662.04,85067.08,85104.24,6871.29,6.000891e+08,87433.23,1,85990.3700,66.675576,187.674224,157.983618,85104.24,85702.44,1,1,0,3
2025-03-02 16:00:00,91954.91,87352.89,87433.23,12097.35,1.088302e+09,91247.12,1,86277.0360,81.452069,589.637638,244.314422,87433.23,85104.24,1,1,0,3
2025-03-02 17:00:00,95106.04,90689.81,91247.12,8378.28,7.791416e+08,94130.91,1,86680.3800,86.372516,1127.892647,421.030067,91247.12,87433.23,1,1,0,3
2025-03-02 18:00:00,94445.48,92346.14,94130.91,4417.80,4.120838e+08,92790.10,1,87014.5385,76.244880,1429.789733,622.782000,94130.91,91247.12,1,1,0,3


#### Handle Continuous Data

In [24]:
df.columns

Index(['high', 'low', 'open', 'volumefrom', 'volumeto', 'close',
       'conversionType', 'MA20', 'RSI', 'MACD', 'MACD_signal', 'Lag_1',
       'Lag_2', 'day', 'year', 'month', 'day_name'],
      dtype='object')

In [25]:
from sklearn.preprocessing import StandardScaler

In [26]:
scaler = StandardScaler()

In [27]:
df[['high', 'low', 'open', 'volumefrom', 'volumeto',
        'MA20', 'RSI', 'MACD', 'MACD_signal']] = scaler.fit_transform(df[['high', 'low', 'open', 'volumefrom', 'volumeto',
        'MA20', 'RSI', 'MACD', 'MACD_signal']])

In [28]:
df.head(2)

,high,low,open,volumefrom,volumeto,close,conversionType,MA20,RSI,MACD,MACD_signal,Lag_1,Lag_2,day,year,month,day_name
time,,,,,,,,,,,,,,,,,
2025-03-02 14:00:00,-0.537592,-0.559831,-0.520361,0.276545,0.237365,85104.24,1,-0.494244,-1.105747,-0.055844,0.12990,85702.44,85980.07,1,1,0,3
2025-03-02 15:00:00,-0.209579,-0.557561,-0.591077,4.787338,4.807142,87433.23,1,-0.482368,1.278101,0.193621,0.14555,85104.24,85702.44,1,1,0,3


#### Define target and predictors

In [29]:
import numpy as np
df['day_sin'] = np.sin(2 * np.pi * df['day_name'] / 7)
df['day_cos'] = np.cos(2 * np.pi * df['day_name'] / 7)

In [30]:
x = df.drop(['close', 'day_name',  'Lag_1',
       'Lag_2'], axis = 1)
y = df['close'].shift(-1)   #This is to make sure that this hours data is predicted by the info from the past hourx

In [31]:
x.shape, y.shape   #After the shift, the both data remains the same length as the last instance would be of type 'Nan'

((1968, 15), (1968,))

#### Split data into train and test

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
x_train, x_test, y_train, y_test = train_test_split(x,y, shuffle=False, random_state = 69, train_size = 0.8)

## Build Model

In [34]:
#import model and dependencies
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

In [35]:
#intialize model and define parameters
modelRF = RandomForestRegressor(n_estimators=100,
    criterion='squared_error',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=5,
    min_weight_fraction_leaf=0.0,
    max_features=1.0,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=30,
    random_state=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,)

#### Fit model

In [36]:
len(x)

1968

In [37]:
modelRF.fit(x_train,y_train)

RandomForestRegressor(min_samples_leaf=5, n_jobs=30)

#### Evaluate model

In [38]:
modelRF.score(x_train,y_train)

0.9939375076009598

In [39]:
print("model performance on train", modelRF.score(x_train, y_train),
      f"\nmodel performance on unseen data",  modelRF.score(x_test[:393], y_test[:393]))

model performance on train 0.9939375076009599 
model performance on unseen data -5.343314240888319


#### Use a simpler model

In [40]:
from sklearn.linear_model import LinearRegression

In [41]:
modelLR = LinearRegression(fit_intercept=True,
    n_jobs=50,
    positive=False,)

In [42]:
modelLR.fit(x_train, y_train)

LinearRegression(n_jobs=50)

In [43]:
len(x_test)

394

In [44]:
print("model performance on train", modelLR.score(x_train, y_train),
      f"\nmodel performance on unseen data",  modelLR.score(x_test[:393], y_test[:393]))

model performance on train 0.9888119618893401 
model performance on unseen data 0.9771786035582464


#### make predictions with model

In [45]:
modelLR.predict(x_test[391:392])

array([109412.74053095])

In [46]:
y_test[391:392]

time
2025-05-23 11:00:00    108454.7
Name: close, dtype: float64

In [47]:
len(x_train.columns)

15